In [1]:
import librosa
import os
import pickle
from scipy.io import wavfile
import numpy as np
import pandas as pd
import random
from pydub import AudioSegment
from tqdm import tqdm

from preprocessing import perform_vad, resample_wav, padding, cut_wav_into_clips, extract_features


c:\Users\jan20\PycharmProjects\PytorchProjects\venv\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
wav_dir = "train\\audio"

## Get full training list

In [40]:
labels_path = 'train\\validation_list.txt'
output_labels_path = 'train\\training_list.txt'
with open(labels_path, "r") as f:
    lines = f.readlines()
file_paths = [line.strip() for line in lines]

In [43]:
with open(output_labels_path, "w") as f1:
    for folder in os.listdir(wav_dir):
        if folder == '_background_noise_' or folder == 'silence':
            continue
        else:
            for file in os.listdir(os.path.join(wav_dir, folder)):
                path = f"{folder}/{file}"
                if path not in file_paths:
                    f1.write(path + '\n')

In [ ]:


with open(output_labels_path', "w") as f1:
    for file in part1:
        f1.write("silence/" + file + "\n")

# Feature extraction

## `unknown` detection / `label` classification

### Training part

In [44]:
labels_path = 'train\\training_list.txt'
with open(labels_path, "r") as f:
    lines = f.readlines()
file_paths_train = [line.strip() for line in lines]


In [53]:
features_train = []

for i, file in tqdm(enumerate(file_paths_train), total=len(file_paths_train), leave=True):
    wav_file = os.path.join(wav_dir,file.split('/')[0],file.split('/')[1])
    wav_file2 = 'working_sample.wav'
    label = file.split('/')[0]

    # Preprocess the data  
    perform_vad(wav_file, wav_file2)
    padding(wav_file2, wav_file2, 1000)
    resample_wav(wav_file2, wav_file2, 8000)

    # Extract features
    features = extract_features(wav_file2)

    # Add to the list
    features_train.append([features, label])

100%|██████████| 57923/57923 [1:12:09<00:00, 13.38it/s]


In [56]:
with open('extracted_features\\features_training.pkl', 'wb') as f:
    pickle.dump(features_train, f)

### Validation part

In [12]:
labels_path = 'train\\validation_list.txt'
with open(labels_path, "r") as f:
    lines = f.readlines()
file_paths_val = [line.strip() for line in lines]

In [13]:
features_val = []

for i, file in tqdm(enumerate(file_paths_val), total=len(file_paths_val), leave=True):
    wav_file = os.path.join(wav_dir,file.split('/')[0],file.split('/')[1],file)
    wav_file2 = 'working_sample.wav'
    label = file.split('/')[0]

    # Preprocess the data  
    perform_vad(wav_file, wav_file2)
    padding(wav_file2, wav_file2, 1000)
    resample_wav(wav_file2, wav_file2, 8000)

    # Extract features
    features = extract_features(wav_file2)

    # Add to the list
    features_train.append([features, label])

  1%|          | 69/6798 [00:04<06:36, 16.96it/s]


KeyboardInterrupt: 

In [4]:
with open('extracted_features\\features_validation.pkl', 'wb') as f:
    pickle.dump(features_val, f)

## `silence` detection

### Silence clips list creation

In [3]:
input_folder = "train\\audio\\_background_noise_"
output_folder = "train\\audio\\silence"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(input_folder):
    if filename.endswith(".wav"):
        cut_wav_into_clips(f"{input_folder}\\{filename}", output_folder, filename, clip_duration_ms=1000)

In [23]:
dir_path = "train\\audio\\silence"

file_list = os.listdir(dir_path)

random.shuffle(file_list)

# Split the list into two parts
split_index = int(len(file_list) * 0.8)
part1 = file_list[:split_index]
part2 = file_list[split_index:]

# Define the output file paths
file1_path = "train\\silence_testing_list.txt"
file2_path = "train\\silence_validation_list.txt"

# Write the paths to the files to the output text files
with open(file1_path, "w") as f1:
    for file in part1:
        f1.write("silence/" + file + "\n")

with open(file2_path, "w") as f2:
    for file in part2:
        f2.write("silence/" + file + "\n")

### Upsample silence to 3000 samples

In [13]:
with open('train\\silence_training_list.txt', 'r') as file:
    lines = file.readlines()
file_paths_train_silence = [line.strip() for line in lines]

In [14]:
len(file_paths_train_silence)

321

In [16]:
input_dir = "train\\audio\\silence"
output_dir = "train\\audio\\silence"
text_file_dir = "train\\silence_training_list.txt"

target_num_clips = 3000
k = 0
n = len(file_paths_train_silence)

while k + n < target_num_clips:
    
    for i, file in enumerate(file_paths_train_silence):
        file_path = os.path.join(wav_dir,file.split('/')[0],file.split('/')[1])
        
        if k + n >= target_num_clips:
            break


        audio, sr = librosa.load(file_path, sr=16000)
        noise = np.random.randn(len(audio))
        noise_level = 0.1
        audio_noise = audio + noise_level * noise
        pitch_shift = np.random.uniform(-100, 100)
        audio_pitch = librosa.effects.pitch_shift(audio_noise, sr=sr, n_steps=pitch_shift/100.0)
        new_filename = f"augmented{k}.wav"
        wavfile.write(os.path.join(output_dir, new_filename), sr, audio_pitch.astype(np.float32))

        with open(text_file_dir, "a") as f:
            f.write("silence/" + new_filename + "\n")
        k += 1

OSError: [Errno 28] No space left on device

In [17]:
len(os.listdir(input_dir))

1945

In [1]:
#open pickle
with open('extracted_features\\silence_detection_validation.pkl', 'rb') as f:
    x = pickle.load(f)
    len(x)

NameError: name 'pickle' is not defined

### Training part

In [3]:
with open('train\\silence_training_list.txt', 'r') as file:
    lines = file.readlines()
file_paths_train_silence = [line.strip() for line in lines]

In [4]:
len(file_paths_train_silence)

1863

In [5]:
features_train_silence = []

for i, file in tqdm(enumerate(file_paths_train_silence), total=len(file_paths_train_silence), leave=True):
    wav_file = os.path.join(wav_dir,file.split('/')[0],file.split('/')[1])
    wav_file2 = 'working_sample.wav'
    label = file.split('/')[0]

    # Preprocess the data  (without VAD)
    padding(wav_file, wav_file2, 1000)
    resample_wav(wav_file2, wav_file2, 8000)

    # Extract features
    features = extract_features(wav_file2)

    # Add to the list
    features_train_silence.append([features, label])

 17%|█▋        | 319/1863 [00:15<00:57, 27.00it/s]c:\Users\jan20\PycharmProjects\PytorchProjects\venv\lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)
 17%|█▋        | 321/1863 [00:15<01:14, 20.72it/s]


FileNotFoundError: [WinError 2] Nie można odnaleźć określonego pliku

In [65]:
with open('extracted_features\\silence_detection_training.pkl', 'wb') as f:
    pickle.dump(features_train + features_train_silence, f)

### Validation part

In [67]:
with open('train\\silence_validation_list.txt', 'r') as file:
    lines = file.readlines() + file.readlines()
file_paths_val_silence = [line.strip() for line in lines]

In [68]:
features_val_silence = []

for i, file in tqdm(enumerate(file_paths_val_silence), total=len(file_paths_val_silence), leave=True):
    wav_file = os.path.join(wav_dir,file.split('/')[0],file.split('/')[1])
    wav_file2 = 'working_sample.wav'
    label = file.split('/')[0]

    # Preprocess the data  (without VAD)
    padding(wav_file, wav_file2, 1000)
    resample_wav(wav_file2, wav_file2, 8000)

    # Extract features
    features = extract_features(wav_file2)

    # Add to the list
    features_val_silence.append([features, label])

100%|██████████| 81/81 [00:03<00:00, 21.96it/s]


In [69]:
with open('extracted_features\\silence_detection_validation.pkl', 'wb') as f:
    pickle.dump(features_val + features_val_silence, f)